In [1]:
import globus_sdk

# Use native app credentials if you have a client ID
CLIENT_ID = "04dc20ba-d8e4-4a9a-9cc9-3ed3f3d49447"

SCOPES = ["urn:globus:auth:scope:search.api.globus.org:search"]

auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
auth_client.oauth2_start_flow(
    requested_scopes=SCOPES,
    refresh_tokens=True
)

authorize_url = auth_client.oauth2_get_authorize_url()
print("Go to this URL and login:", authorize_url)

auth_code = input("Enter the authorization code here: ").strip()
token_response = auth_client.oauth2_exchange_code_for_tokens(auth_code)



Go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=04dc20ba-d8e4-4a9a-9cc9-3ed3f3d49447&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=urn%3Aglobus%3Aauth%3Ascope%3Asearch.api.globus.org%3Asearch&state=_default&response_type=code&code_challenge=Kr1qw6gmqD0TZzXpJIjOWM6GeEpvCBKvz1y-Vn7c1qc&code_challenge_method=S256&access_type=offline


Enter the authorization code here:  VnenEPjD48SMTjxOCGFa40VKoUxxoN


In [2]:
# Extract the access token for the Search service
search_tokens = token_response.by_resource_server["search.api.globus.org"]
search_authorizer = globus_sdk.AccessTokenAuthorizer(search_tokens["access_token"])

# Initialize the SearchClient with the authorizer
search_client = globus_sdk.SearchClient(authorizer=search_authorizer)

In [6]:
INDEX_ID = "6871e83e-866b-41bc-8430-e3cf83b43bdc"
QUERY = "I077_SiO2_071822_v58_Exp2_SteadyShear_02_seg2_Preshear300_XPCS_03_009_att03_Lq1_001_0001-5000"  # Match all entries (or a specific keyword)

response = search_client.post_search(
    index_id=INDEX_ID,
    data={"q": QUERY, "limit": 100}
)
print(response)

{
  "total": 7442,
  "gmeta": [
    {
      "@datatype": "GMetaResult",
      "@version": "2019-08-27",
      "subject": "globus://74defd5b-5f61-42fc-bcc4-834c9f376a4f/XPCSDATA/Automate/2022-2/xmlin202207/I077_SiO2_071822_v58_Exp2_SteadyShear_02_seg2_Preshear300_XPCS_03_009_att03_Lq1_001_0001-5000",
      "entries": [
        {
          "content": {
            "files": [
              {
                "filename": "boost_corr.log",
                "sha256": "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855",
                "field_metadata": {},
                "mime_type": "text/x-log",
                "length": 0,
                "url": "https://g-f6125.fd635.8443.data.globus.org/XPCSDATA/Automate/2022-2/xmlin202207/I077_SiO2_071822_v58_Exp2_SteadyShear_02_seg2_Preshear300_XPCS_03_009_att03_Lq1_001_0001-5000/boost_corr.log",
                "md5": "d41d8cd98f00b204e9800998ecf8427e"
              },
              {
                "filename": "I077_SiO2_071822_v58_Ex

In [7]:


keys_set = set()
for entry in response["gmeta"]:
    for content in entry.get("content", []):
        keys_set.update(content.keys())

print(f"\n🔑 Keys found in index {INDEX_ID}:")
for key in sorted(keys_set):
    print(f"- {key}")


🔑 Keys found in index 6871e83e-866b-41bc-8430-e3cf83b43bdc:


In [24]:
for k in range(0,len(response["gmeta"])):
    print(response['gmeta'][k]["subject"])
    print(response['gmeta'][k]["entries"][0])

globus://74defd5b-5f61-42fc-bcc4-834c9f376a4f/XPCSDATA/Automate/2022-2/xmlin202207/I077_SiO2_071822_v58_Exp2_SteadyShear_02_seg2_Preshear300_XPCS_03_009_att03_Lq1_001_0001-5000
{'content': {'files': [{'filename': 'boost_corr.log', 'sha256': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855', 'field_metadata': {}, 'mime_type': 'text/x-log', 'length': 0, 'url': 'https://g-f6125.fd635.8443.data.globus.org/XPCSDATA/Automate/2022-2/xmlin202207/I077_SiO2_071822_v58_Exp2_SteadyShear_02_seg2_Preshear300_XPCS_03_009_att03_Lq1_001_0001-5000/boost_corr.log', 'md5': 'd41d8cd98f00b204e9800998ecf8427e'}, {'filename': 'I077_SiO2_071822_v58_Exp2_SteadyShear_02_seg2_Preshear300_XPCS_03_009_att03_Lq1_001_0001-5000.hdf', 'sha256': '07955e6e6159c65e56eb07e317a21f4188fdd36c5fb8a0374049f0b7232c0f01', 'field_metadata': {}, 'mime_type': 'application/x-hdf', 'length': 46520, 'url': 'https://g-f6125.fd635.8443.data.globus.org/XPCSDATA/Automate/2022-2/xmlin202207/I077_SiO2_071822_v58_Exp2_SteadyS

In [9]:
index_list_response = search_client.index_list()
print("Available Indexes:")
for idx in index_list_response["index_list"]:
    print(f"- Display Name: {idx['display_name']}")
    print(f"  Index ID: {idx['id']}\n")

SearchAPIError: ('GET', 'https://search.api.globus.org/v1/index_list', 'Bearer', 403, 'Forbidden.InsufficientScopes', 'index_list request denied by service', '95a59336d22a4a6f9295e6239bdd7dc5')